# SSVEP 腦波分析完整教學
## Steady-State Visual Evoked Potential 穩態視覺誘發電位

### 營隊教學用

---

## 課程大綱
1. SSVEP 原理介紹
2. 數據載入與視覺化
3. 訊號預處理
4. 特徵提取
5. 機器學習分類
6. 結果評估

---

## 1. SSVEP 原理簡介

### 什麼是 SSVEP？
- 當我們看著特定頻率閃爍的光源時，大腦視覺皮層會產生相同頻率的電位反應
- 例如：看著 10Hz 閃爍的燈光，腦波中會出現 10Hz 的訊號
- 應用：腦機介面(BCI)、拼字器、輪椅控制等

### 常見刺激頻率：
- 8Hz, 10Hz, 12Hz, 15Hz 等
- 通常使用 4-6 個不同頻率作為控制指令


In [ ]:
# 安裝必要套件
!pip install -q mne

print("套件安裝完成")

In [ ]:
# 導入所需函式庫
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import signal
from scipy.fft import fft, fftfreq
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import warnings
warnings.filterwarnings('ignore')

# 設定繪圖風格
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)

print("所有套件載入完成")
print(f"NumPy 版本: {np.__version__}")

## 2. 產生模擬 SSVEP 數據

為了教學方便，我們產生模擬的 SSVEP 訊號

In [ ]:
# 實驗參數設定
sampling_rate = 250  # 取樣頻率 (Hz)
duration = 4  # 每個 trial 的時間長度 (秒)
n_trials = 40  # 每個頻率的 trial 數量
n_channels = 8  # EEG 通道數

# SSVEP 刺激頻率
target_freqs = [8, 10, 12, 15]  # Hz
n_classes = len(target_freqs)

print(f"實驗設定：")
print(f"  取樣頻率: {sampling_rate} Hz")
print(f"  Trial 長度: {duration} 秒")
print(f"  每類別 Trial 數: {n_trials}")
print(f"  刺激頻率: {target_freqs} Hz")
print(f"  總共數據: {n_trials * n_classes} trials")

In [ ]:
def generate_ssvep_signal(freq, sampling_rate, duration, n_channels=8, snr=0.5):
    """
    產生模擬 SSVEP 訊號
    
    參數:
        freq: 目標頻率
        sampling_rate: 取樣頻率
        duration: 訊號長度(秒)
        n_channels: 通道數
        snr: 訊噪比 (0-1 之間，越大訊號越乾淨)
    """
    n_samples = int(sampling_rate * duration)
    t = np.linspace(0, duration, n_samples)
    
    # 產生多通道訊號
    signal_data = np.zeros((n_channels, n_samples))
    
    for ch in range(n_channels):
        # 基礎 SSVEP 訊號 (基頻 + 二次諧波)
        ssvep = (snr * np.sin(2 * np.pi * freq * t + np.random.rand()) + 
                 snr * 0.3 * np.sin(2 * np.pi * 2 * freq * t + np.random.rand()))
        
        # 加入背景腦波 (alpha 波 8-13Hz)
        alpha = 0.3 * np.sin(2 * np.pi * (9 + np.random.rand()) * t)
        
        # 雜訊
        noise = (1 - snr) * np.random.randn(n_samples) * 0.5
        
        # 組合訊號
        signal_data[ch] = ssvep + alpha + noise
        
        # 後端通道訊號較強 (模擬視覺皮層位置)
        if ch >= n_channels - 3:
            signal_data[ch] *= 1.5
    
    return signal_data

# 產生所有數據
X_data = []
y_labels = []

print("產生模擬 SSVEP 數據...")
for class_idx, freq in enumerate(target_freqs):
    for trial in range(n_trials):
        signal_data = generate_ssvep_signal(freq, sampling_rate, duration, n_channels)
        X_data.append(signal_data)
        y_labels.append(class_idx)

X_data = np.array(X_data)  # shape: (n_trials_total, n_channels, n_samples)
y_labels = np.array(y_labels)

print(f"\n數據產生完成")
print(f"數據形狀: {X_data.shape}")
print(f"標籤形狀: {y_labels.shape}")

## 3. 數據視覺化

讓我們看看 SSVEP 訊號長什麼樣子

In [ ]:
# 繪製不同頻率的原始訊號
fig, axes = plt.subplots(2, 2, figsize=(15, 8))
axes = axes.flatten()

time_axis = np.linspace(0, duration, X_data.shape[2])

for idx, freq in enumerate(target_freqs):
    # 取得該頻率的第一個 trial
    trial_data = X_data[idx * n_trials]  # shape: (n_channels, n_samples)
    
    # 繪製所有通道
    for ch in range(n_channels):
        axes[idx].plot(time_axis, trial_data[ch] + ch*3, alpha=0.7, linewidth=0.8)
    
    axes[idx].set_title(f'Stimulus Frequency: {freq} Hz', fontsize=14, fontweight='bold')
    axes[idx].set_xlabel('Time (sec)', fontsize=11)
    axes[idx].set_ylabel('Channel (offset)', fontsize=11)
    axes[idx].grid(True, alpha=0.3)
    axes[idx].set_xlim([0, 2])  # 只顯示前 2 秒

plt.tight_layout()
plt.suptitle('SSVEP Raw Signals - Different Stimulus Frequencies', fontsize=16, y=1.02)
plt.show()

print("圖說明: 這是不同刺激頻率下的原始 SSVEP 訊號")
print("可以看到每個通道的腦波活動，後端通道(編號較大)訊號較強")

In [ ]:
# 頻譜分析 - 看看頻域特徵
def plot_spectrum(signal_data, sampling_rate, freq_label):
    """計算並繪製功率頻譜"""
    # 使用後端通道 (視覺皮層)
    channel_data = signal_data[-1]  # 最後一個通道
    
    # FFT
    n = len(channel_data)
    fft_vals = fft(channel_data)
    fft_freq = fftfreq(n, 1/sampling_rate)
    
    # 只取正頻率
    positive_freq_idx = fft_freq > 0
    fft_freq = fft_freq[positive_freq_idx]
    fft_power = np.abs(fft_vals[positive_freq_idx])
    
    return fft_freq, fft_power

# 繪製頻譜
fig, axes = plt.subplots(2, 2, figsize=(15, 8))
axes = axes.flatten()

for idx, freq in enumerate(target_freqs):
    trial_data = X_data[idx * n_trials]
    fft_freq, fft_power = plot_spectrum(trial_data, sampling_rate, freq)
    
    axes[idx].plot(fft_freq, fft_power, linewidth=1.5)
    axes[idx].axvline(freq, color='red', linestyle='--', linewidth=2, 
                     label=f'Target {freq}Hz')
    axes[idx].axvline(freq*2, color='orange', linestyle='--', linewidth=1.5, 
                     label=f'2nd Harmonic {freq*2}Hz', alpha=0.7)
    
    axes[idx].set_xlim([0, 40])
    axes[idx].set_title(f'Frequency Spectrum: {freq} Hz Stimulus', fontsize=14, fontweight='bold')
    axes[idx].set_xlabel('Frequency (Hz)', fontsize=11)
    axes[idx].set_ylabel('Power', fontsize=11)
    axes[idx].legend()
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.suptitle('SSVEP Frequency Analysis - Clear Target Frequency Peaks', fontsize=16, y=1.02)
plt.show()

print("圖說明: 這是 SSVEP 訊號的頻譜分析")
print("可以明顯看到目標頻率和二次諧波的峰值")

## 4. 訊號預處理

預處理步驟:
1. 帶通濾波 (5-40Hz) - 保留 SSVEP 相關頻段
2. 標準化

In [ ]:
def preprocess_signal(data, sampling_rate, lowcut=5, highcut=40):
    """
    預處理 SSVEP 訊號
    
    參數:
        data: 輸入訊號 (n_channels, n_samples)
        sampling_rate: 取樣頻率
        lowcut: 低頻截止
        highcut: 高頻截止
    """
    # 設計帶通濾波器
    nyquist = sampling_rate / 2
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = signal.butter(4, [low, high], btype='band')
    
    # 對每個通道進行濾波
    filtered_data = np.zeros_like(data)
    for ch in range(data.shape[0]):
        filtered_data[ch] = signal.filtfilt(b, a, data[ch])
    
    # 標準化 (每個通道)
    for ch in range(filtered_data.shape[0]):
        filtered_data[ch] = (filtered_data[ch] - np.mean(filtered_data[ch])) / np.std(filtered_data[ch])
    
    return filtered_data

# 預處理所有數據
print("預處理數據中...")
X_preprocessed = np.array([preprocess_signal(trial, sampling_rate) for trial in X_data])

print(f"預處理完成")
print(f"處理後數據形狀: {X_preprocessed.shape}")

In [ ]:
# 比較預處理前後的訊號
fig, axes = plt.subplots(2, 1, figsize=(15, 8))

sample_trial = 0
sample_channel = -1  # 最後一個通道
time_axis = np.linspace(0, duration, X_data.shape[2])

# 原始訊號
axes[0].plot(time_axis, X_data[sample_trial, sample_channel], linewidth=1)
axes[0].set_title('Before Preprocessing - Raw Signal', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Time (sec)')
axes[0].set_ylabel('Amplitude')
axes[0].grid(True, alpha=0.3)

# 預處理後訊號
axes[1].plot(time_axis, X_preprocessed[sample_trial, sample_channel], linewidth=1, color='green')
axes[1].set_title('After Preprocessing - Filtered + Normalized', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Time (sec)')
axes[1].set_ylabel('Amplitude')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("圖說明: 比較預處理前後的訊號")
print("預處理後的訊號更乾淨，雜訊被過濾掉了")

## 5. 特徵提取

我們使用功率譜密度 (Power Spectral Density, PSD) 作為特徵

方法: 計算每個目標頻率及其諧波的功率

In [ ]:
def extract_features(signal_data, sampling_rate, target_freqs, n_harmonics=2):
    """
    提取 SSVEP 特徵
    
    參數:
        signal_data: 訊號 (n_channels, n_samples)
        sampling_rate: 取樣頻率
        target_freqs: 目標頻率列表
        n_harmonics: 使用的諧波數量
    
    回傳:
        features: 特徵向量
    """
    features = []
    
    # 對每個通道計算 PSD
    for ch in range(signal_data.shape[0]):
        # 計算功率譜密度
        freqs, psd = signal.welch(signal_data[ch], sampling_rate, nperseg=sampling_rate*2)
        
        # 對每個目標頻率
        for target_freq in target_freqs:
            # 基頻和諧波的功率
            for harmonic in range(1, n_harmonics + 1):
                freq_of_interest = target_freq * harmonic
                
                # 找到最接近的頻率索引
                idx = np.argmin(np.abs(freqs - freq_of_interest))
                
                # 取該頻率附近的平均功率 (±0.5Hz)
                freq_band = (freqs >= freq_of_interest - 0.5) & (freqs <= freq_of_interest + 0.5)
                power = np.mean(psd[freq_band])
                
                features.append(power)
    
    return np.array(features)

# 提取所有 trial 的特徵
print("提取特徵中...")
X_features = np.array([extract_features(trial, sampling_rate, target_freqs) 
                       for trial in X_preprocessed])

print(f"特徵提取完成")
print(f"特徵矩陣形狀: {X_features.shape}")
print(f"每個 trial 的特徵數: {X_features.shape[1]}")

In [ ]:
# 視覺化特徵分布
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.flatten()

for class_idx, freq in enumerate(target_freqs):
    # 取得該類別的所有特徵
    class_features = X_features[y_labels == class_idx]
    
    # 繪製前 16 個特徵的分布
    axes[class_idx].boxplot(class_features[:, :16], labels=range(1, 17))
    axes[class_idx].set_title(f'Class {class_idx}: {freq} Hz - Feature Distribution', 
                             fontsize=13, fontweight='bold')
    axes[class_idx].set_xlabel('Feature Number')
    axes[class_idx].set_ylabel('Feature Value')
    axes[class_idx].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("圖說明: 不同類別的特徵分布")
print("可以看到不同頻率的 SSVEP 訊號有不同的特徵模式")

## 6. 機器學習分類

我們使用支持向量機 (Support Vector Machine, SVM) 來辨識 SSVEP 頻率

In [ ]:
# 分割訓練集和測試集
X_train, X_test, y_train, y_test = train_test_split(
    X_features, y_labels, test_size=0.3, random_state=42, stratify=y_labels
)

print(f"訓練集大小: {X_train.shape[0]} trials")
print(f"測試集大小: {X_test.shape[0]} trials")
print(f"\n訓練集標籤分布: {np.bincount(y_train)}")
print(f"測試集標籤分布: {np.bincount(y_test)}")

In [ ]:
# 訓練 SVM 分類器
print("訓練 SVM 分類器...")

clf = SVC(kernel='rbf', C=1.0, gamma='scale')
clf.fit(X_train, y_train)

# 預測
y_pred = clf.predict(X_test)

# 計算準確率
accuracy = accuracy_score(y_test, y_pred)

print(f"\nSVM 準確率: {accuracy*100:.2f}%")
print(f"\n這表示模型能夠正確辨識 {accuracy*100:.2f}% 的 SSVEP 頻率！")

## 7. 結果評估與視覺化

In [ ]:
# 繪製混淆矩陣
fig, ax = plt.subplots(figsize=(8, 6))

class_names = [f'{freq}Hz' for freq in target_freqs]
cm = confusion_matrix(y_test, y_pred)

# 正規化到 0-1
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

# 繪製熱圖
sns.heatmap(cm_normalized, annot=True, fmt='.2f', cmap='Blues', 
            xticklabels=class_names, yticklabels=class_names,
            ax=ax, cbar_kws={'label': 'Proportion'})

ax.set_title(f'Confusion Matrix\nAccuracy: {accuracy*100:.2f}%', 
            fontsize=13, fontweight='bold')
ax.set_ylabel('True Label', fontsize=11)
ax.set_xlabel('Predicted Label', fontsize=11)

plt.tight_layout()
plt.show()

print("圖說明: 混淆矩陣 (Confusion Matrix)")
print("對角線上的數值越高越好，代表預測正確")
print("非對角線的數值代表分類錯誤")

In [ ]:
# 詳細分類報告
print("="*60)
print("分類報告 (Classification Report)")
print("="*60)

print(classification_report(y_test, y_pred, 
                          target_names=class_names,
                          digits=4))

print("\n指標說明:")
print("  Precision (精確率): 預測為某類別中，實際正確的比例")
print("  Recall (召回率): 實際為某類別中，被正確預測的比例")
print("  F1-score: Precision 和 Recall 的調和平均")
print("  Support: 該類別在測試集中的樣本數")

## 8. 課程總結

### 我們學到了什麼？

1. **SSVEP 原理**: 大腦視覺皮層對特定頻率閃爍光的反應

2. **完整 Pipeline**:
   - 數據採集與視覺化
   - 訊號預處理（濾波、標準化）
   - 特徵提取（功率譜密度）
   - 分類器訓練與評估

3. **機器學習方法**:
   - 支持向量機 (SVM)
   - 訓練/測試集分割
   - 性能評估

### 實際應用
- 腦機介面 (BCI)
- 輔助溝通系統
- 遊戲控制
- 醫療復健

### 延伸學習
- 嘗試使用真實 EEG 數據集
- 探索深度學習方法 (CNN, LSTM)
- 優化特徵提取方法
- 實時 SSVEP 系統開發


## 9. 互動練習區

讓學員自己嘗試修改參數，觀察結果變化：

In [ ]:
# 練習 1: 改變訊噪比，觀察對分類性能的影響
# 提示: 修改 generate_ssvep_signal 函數中的 snr 參數（目前是 0.5）
# 試試看 snr=0.3 (更多雜訊) 或 snr=0.8 (更乾淨的訊號)

# 練習 2: 嘗試增加更多目標頻率
# 提示: 修改 target_freqs = [8, 10, 12, 15]
# 試試看加入 6Hz 和 14Hz: target_freqs = [6, 8, 10, 12, 14, 15]

# 練習 3: 調整濾波器參數，看看效果如何
# 提示: 修改 preprocess_signal 中的 lowcut=5 和 highcut=40
# 試試看 lowcut=8, highcut=30

# 練習 4: 嘗試使用不同數量的訓練數據
# 提示: 修改 n_trials = 40
# 試試看 n_trials=20 (較少數據) 或 n_trials=60 (較多數據)

print("請在上面的 code cells 中嘗試修改參數")
print("修改後重新執行所有 cells，觀察準確率的變化！")